                                    CUSTOMER SEGMENTATION USING MACHINE LEARNING(UNSUPERVISED LEARNING)

In [2]:
#IMPORTING LIBRARIES FOR HANDLING AND GENERATING FAKE DATAFRAME
import numpy as np
import pandas as pd
import random
from faker import Faker

In [ ]:
#GENERTING FAKE DATA
fake = Faker()

num_customers = 2500
max_transactions_per_customer = 20 

customers = [f"CUST{str(i).zfill(6)}" for i in range(1, num_customers + 1)]

transaction_data = []
for customer_id in customers:
    num_transactions = random.randint(1, max_transactions_per_customer)
    for _ in range(num_transactions):
        transaction_data.append({
            "customer_id": customer_id,
            "transaction_id": f"TRANS{str(len(transaction_data) + 1).zfill(6)}",
            "transaction_amount": np.random.uniform(10, 1000),
            "transaction_type": random.choice(["deposit", "withdrawal"]),
            "transaction_date": fake.date_between(start_date='-1y', end_date='today'),
        })

transaction_data = pd.DataFrame(transaction_data)


In [ ]:
#DISPLAY THE DATAFRAME
transaction_data.head()

In [ ]:
#SAVE THE FAKE DATASET IN CSV FILE
transaction_data.to_csv("Loan Transaction data.csv", index=False)


In [3]:
#READ THE FAKE DATA 
transaction_data = pd.read_csv(r"D:\BANK PROJECT\Loan Transaction data.csv")


In [ ]:
#DISPLAY THE DATAFRAME
transaction_data.head()

                                                  DATA CLEANING

In [ ]:
#CHECKING THE TOTAL TRANSACTION AND COUNT THE TOTAL CUSTOMER 
print(f"Total transactions: {len(transaction_data)}")
print(f"Unique customers: {transaction_data['customer_id'].nunique()}")

In [ ]:
#CHECKING THE NULL VALES 
transaction_data.isna().sum()

In [ ]:
#CHECKING DUPLICATE VALUE
transaction_data.duplicated().sum()

In [ ]:
#CHECK THE INFO OF DATAFRAME
transaction_data.info()

In [ ]:
#DESCRIBE THE DATAFRAME 
transaction_data.describe()

In [ ]:
#CHECKING THE ALL COLUMNS COUNTS
transaction_data.count()

In [ ]:
#CHECK THE DATATYPE OF THE DATAFRAME
transaction_data.dtypes

In [ ]:
#CHECK THE VALUE COUNT IN TRANSACTIO TYPE COLUMN
transaction_data['transaction_type'].value_counts()

In [16]:
#IMPORTING LIBRARIES FOR EDA
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#CHECKING THE OUTLIER
plt.figure(figsize=(6, 6))
sns.boxplot(data=transaction_data)

In [4]:
#CONVERTING THE COLUMN TO CORRECT DATATYPE
transaction_data['transaction_date'] = pd.to_datetime(transaction_data['transaction_date'])

In [5]:
#CONVERTING THE RUPEES TO DOLLAR
transaction_data["transaction_amount"] = transaction_data["transaction_amount"] / 86.66


In [6]:
#ROUNDING THE COLUMNS VALUES
transaction_data['transaction_amount'] = transaction_data['transaction_amount'].round(1).astype(float)

In [ ]:
#DISPLAY THE DATAFRAME
transaction_data.head()

In [7]:
#CREATING NEW COLUMNS
elements = ['withdrawal','deposit']

for element in elements:
    Tranformed_Columns = transaction_data['transaction_type'] == element
    transaction_data[f'{element}_Value'] = transaction_data['transaction_amount'].where(Tranformed_Columns,None)

In [8]:
#FILLING THE NULL WITH 0 FOR SUM IT
transaction_data.fillna(0,inplace=True)

In [ ]:
#GROUPING THE COLUMNS FOR TRAINING THE MODEL MORE EFFICIENTLY
customer_data = transaction_data.groupby("customer_id").agg(
    total_no_of_transactions=("transaction_id", "count"),
    total_amount_in_dollar=("transaction_amount", "sum"),
    num_of_deposits=("transaction_type", lambda x: (x == "deposit").sum()),
    num_of_withdrawals=("transaction_type", lambda x: (x == "withdrawal").sum()),
    withdrawals_amount_in_dollar=("withdrawal_Value",'sum'),
    deposits_amount_in_dollar=("deposit_Value",'sum')
).reset_index()
customer_data.head()

In [10]:
customer_data["balance_amount_in_dollar"] = customer_data["deposits_amount_in_dollar"] - customer_data["withdrawals_amount_in_dollar"]

In [ ]:
customer_data.head()

In [12]:
customer_data.rename(columns={"total_amount_in_dollar": "total_transaction_amount_in_dollar"}, inplace=True)


In [ ]:
#AGAIN CHECK THE DATATYPE AFTER GROUPING
customer_data.dtypes

In [ ]:
#CHECKING THE NULL VALUE AFTER GROUPING
customer_data.isna().sum()

In [ ]:
customer_data.head()

                                             EXPLORATORY DATA ANALYSIS

In [ ]:
# SELECT TOP 10 CUSTOMERS BASED ON TOTAL AMOUNT IN DESCENDING ORDER
top_10 = customer_data[["customer_id","total_no_of_transactions","total_transaction_amount_in_dollar"]].sort_values(by='total_transaction_amount_in_dollar', ascending=False).head(10)
plt.figure(figsize=(13, 5))
sns.barplot(y="customer_id", x="total_transaction_amount_in_dollar", data=top_10, orient='h')
plt.xlabel("total_transaction_amount_in_dollar")
plt.ylabel("Customer ID")
plt.title("Top 10 Customers by Total Amount")
plt.show()

In [ ]:
#SELECT TOP 10 CUSTOMERS BASED ON WITHDRAWALS AMOUNT 
top_10 = customer_data[["customer_id","withdrawals_amount_in_dollar"]].sort_values(by='withdrawals_amount_in_dollar', ascending=False).head(10)

plt.figure(figsize=(13, 5))
sns.barplot(x="customer_id", y="withdrawals_amount_in_dollar", data=top_10, palette="coolwarm")
plt.title("Top 10 Customers by Withdrawals Amount", fontsize=16, fontweight="bold")
plt.xlabel("Customer ID", fontsize=14)
plt.ylabel("Withdrawals Amount", fontsize=14)
plt.show()

In [ ]:
# CREATE A DISTRIBUTION PLOT FOR TOTAL AMOUNT WITH KDE
sns.displot(data=customer_data, x="total_transaction_amount_in_dollar", kde=True)  
plt.show()


In [ ]:
# SELECT TOP 10 CUSTOMERS BASED ON DEPOSITS AMOUNT 
top_10 = customer_data[["customer_id","deposits_amount_in_dollar"]].sort_values(by='deposits_amount_in_dollar', ascending=False).head(10)

plt.figure(figsize=(13, 5))
sns.barplot(x="customer_id", y="deposits_amount_in_dollar", data=top_10, palette="Greens")
plt.title("Top 10 Customers by Deposits Amount", fontsize=16, fontweight="bold")
plt.xlabel("Customer ID", fontsize=14)
plt.ylabel("Deposits Amount", fontsize=14)
plt.show()

In [ ]:
#CHECKING THE CORRELATION FOR NUMERIC COLUMNS USING HEATMAP
correlation_matrix = customer_data[["total_no_of_transactions",
                                    "num_of_deposits", "num_of_withdrawals", "withdrawals_amount_in_dollar", "deposits_amount_in_dollar","total_transaction_amount_in_dollar","balance_amount_in_dollar"]].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Matrix")
plt.show()

In [ ]:
customer_data.head()

In [ ]:
#CHECKING OUTLIER FOR ALL THE COLUMNS
plt.figure(figsize=(16, 5))
sns.boxplot(data = customer_data)

In [ ]:
Q1 = customer_data['deposits_amount_in_dollar'].quantile(0.25)
Q3 = customer_data['deposits_amount_in_dollar'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Determine the upper and lower bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = customer_data[(customer_data['deposits_amount_in_dollar'] < lower_bound) | (customer_data['deposits_amount_in_dollar'] > upper_bound)]

# Display results
print("Lower bound:", lower_bound)
print("Upper bound:", upper_bound)
outliers


In [ ]:
Q1 = customer_data['withdrawals_amount_in_dollar'].quantile(0.25)
Q3 = customer_data['withdrawals_amount_in_dollar'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Determine the upper and lower bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = customer_data[(customer_data['withdrawals_amount_in_dollar'] < lower_bound) | (customer_data['withdrawals_amount_in_dollar'] > upper_bound)]

# Display results
print("Lower bound:", lower_bound)
print("Upper bound:", upper_bound)
outliers


In [171]:
# Remove outliers
customer_data = customer_data[
    (customer_data['withdrawals_amount_in_dollar'] >= lower_bound) & 
    (customer_data['withdrawals_amount_in_dollar'] <= upper_bound)
]


In [172]:
# Remove outliers
customer_data = customer_data[
    (customer_data['deposits_amount_in_dollar'] >= lower_bound) & 
    (customer_data['deposits_amount_in_dollar'] <= upper_bound)
]



In [ ]:
customer_data.head()

In [175]:
#SAVE THE DATASET AFTER CLEANING
customer_data.to_csv("cleaned_loan_dataset.csv", index=False)


In [176]:
#READ THE CLEANED DATASET
customer_data = pd.read_csv("d:/BANK PROJECT/cleaned_loan_dataset.csv")

In [ ]:
#DISPLAY THE DATAFRAME 
customer_data.head()

In [ ]:
#CHECK THE DATATYPE AFTER ALL CLEANING DONE
customer_data.dtypes

In [ ]:
                                   DATA PREPROCESSING AND MODEL TRAINING

In [179]:
#IMPORTED ALL THE LIBRARIES FOR TRAINING,EVALUATE THE MODLE AND EVALUATE THE MODEL
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from imblearn.over_sampling import KMeansSMOTE
from sklearn.decomposition import PCA

from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA


In [ ]:
#FEATURE ENCODE THE TRAINING DATA COLUMNS
features = customer_data[["total_no_of_transactions", "total_transaction_amount_in_dollar", "num_of_deposits", 
                          "num_of_withdrawals", "withdrawals_amount_in_dollar", "deposits_amount_in_dollar"]]

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

scaled_customer_data = pd.DataFrame(scaled_features, columns=features.columns.tolist())
scaled_customer_data.head()

In [189]:
# TRAIN THE DATA USING KMEANS CLUSTERING
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10, max_iter=300, algorithm='lloyd')

customer_data['cluster'] = kmeans.fit_predict(scaled_customer_data)

In [ ]:
#HYPERTUNING THE CENTROID BASED K MEANS CLUSTERING MODEL 
kmeans = KMeans(random_state=42)

param_grid = {
    'n_clusters': np.arange(2, 11), 
    'init': ['k-means++', 'random'],  
    'max_iter': [300, 500, 1000],  
    'n_init': [10, 15],  
    'tol': [1e-4, 1e-3],  
}

grid_search = GridSearchCV(kmeans, param_grid, cv=3, verbose=1, n_jobs=-1)
grid_search.fit(scaled_customer_data)

print("Best Parameters:", grid_search.best_params_)

best_kmeans = KMeans(**grid_search.best_params_, random_state=42)
customer_data["cluster"] = best_kmeans.fit_predict(scaled_customer_data)



In [ ]:
#USING THE ELBOW METHOD CHECKED THE BEST NUMBER OF CLUSTER
inertia = []
for k in range(2, 11):  
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_customer_data)
    inertia.append(kmeans.inertia_)

plt.plot(range(2, 11), inertia, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.show()


In [ ]:
# DISPLAY THE DATAFRAME AFTER CLUSTERING
customer_data.head()

In [ ]:
#CHECKED THE SCORE FOR KMEANS CLUSTERING MODEL
silhouette = silhouette_score(scaled_customer_data, customer_data["cluster"])
db_index = davies_bouldin_score(scaled_customer_data, customer_data["cluster"])

print(f"Silhouette Score: {round(silhouette, 2)}")
print(f"Davies-Bouldin Index: {db_index}")


In [ ]:
#DISPLAY THE PCA PLOT KMEANS MODEL
pca = PCA(n_components=2)
data_pca = pca.fit_transform(scaled_features)

plt.scatter(data_pca[:, 0], data_pca[:, 1], c=customer_data["cluster"], cmap='viridis', s=50)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='red', marker='X', s=200)
plt.title('Cluster Visualization (PCA)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()

In [192]:
#IMPORTING JOBLIB TO SAVE THE MODEL
import joblib

In [ ]:
#SAVED THE GOOD SCORE MODEL
joblib.dump(kmeans, r"d:\BANK PROJECT\kmeans.pkl")

In [ ]:
#SAVE THE SCALED DATA 
joblib.dump(scaled_features, r"d:\BANK PROJECT\scaled_d.pkl") 

In [129]:
#TRAINING THE DENSITY BASED DBSCAN MODEL
dbscan = DBSCAN(eps=0.5, min_samples=5)  
customer_data["cluster"] = dbscan.fit_predict(scaled_features)

In [ ]:
#CHECK THE SCORE FOR DBSCAN MODEL
if len(set(customer_data["cluster"])) > 1:
    silhouette = silhouette_score(scaled_features, customer_data["cluster"])
    db_index = davies_bouldin_score(scaled_features, customer_data["cluster"])
    print(f"Silhouette Score: {silhouette}")
    print(f"Davies-Bouldin Index: {db_index}")
else:
    print("DBSCAN found less than two clusters; Silhouette Score is not defined.")

In [ ]:
#DISPLAY THE PCA PLOT FOR DBSCAN MODEL
pca = PCA(n_components=2)
data_pca = pca.fit_transform(scaled_features)

plt.scatter(data_pca[:, 0], data_pca[:, 1], c=customer_data["cluster"], cmap='viridis', s=50)
plt.title('Cluster Visualization (DBSCAN, PCA)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()

In [ ]:
#TRAINED THE AGGLOMERATIVE CLUSTERING

hc = AgglomerativeClustering(n_clusters=2)
scaled_customer_data['cluster'] = hc.fit_predict(scaled_customer_data)

sch.dendrogram(sch.linkage(scaled_customer_data, method='ward'))

scaled_customer_data.head()

In [ ]:
#CHECK THE SCORE FOR AGGLOMERATIVE CLUSTERING
silhouette = silhouette_score(scaled_customer_data, scaled_customer_data["cluster"])
db_index = davies_bouldin_score(scaled_customer_data, scaled_customer_data["cluster"])

print(f"Silhouette Score: {round(silhouette, 2)}")
print(f"Davies-Bouldin Index: {db_index}")

In [ ]:
#DISPLAY THE PCA PLOT FOR AGGLOMERATIVE CLUSTERING
pca = PCA(n_components=2)
data_pca = pca.fit_transform(scaled_features)

plt.scatter(data_pca[:, 0], data_pca[:, 1], c=scaled_customer_data["cluster"], cmap='viridis', s=50)
plt.title('Cluster Visualization (PCA)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()